In [75]:
import pandas as pd
from pathlib import Path
import glob
import numpy as np
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

In [76]:
ac_df = pd.read_csv('all/dimFacility.csv', sep=',', encoding='cp1252')
ac_df = ac_df.rename(columns=str.lower)

In [77]:
ac_df['acquisitiondate']

0       20010201
1       20060901
2       20031001
3       20090201
4       20020701
          ...   
1236    20210501
1237    20190812
1238    20210701
1239    20210901
1240    20210901
Name: acquisitiondate, Length: 1241, dtype: int64

In [78]:
map_df = pd.read_csv('all/Location.csv')
map_df[map_df['CCN'] == 455754]

,Location_WD_ID,Location_GL_ID,Location_4Digit_ID,Location_Name,Location_ShortName,WD_ID,PCC_ID,ROX_SiteOfService_Key,ROX_SiteOfService_ID,State_ID,...,Address,City,State_Code,Zip_Code,Region,Market,Cluster,Created_Date,Updated_Date,isDeleted
0,10100,101,2401,Northeast Rehabilitation and Healthcare Center,Northeast,777cbbb301d9102389c0717301666136,56.0,92.0,395.0,5147,...,603 Corrine,San Antonio,TX,78218,Texas,South Central Keystone,Missions,2021-08-19 11:03:29.870,2021-09-03 06:15:11.127,False


In [79]:
map_df = map_df[map_df['PCC_ID'].notna()]
map_df = map_df[map_df['CCN'].notna()]
map_duplicated_df = map_df[map_df.duplicated('CCN', keep=False)]
map_duplicated_df[['CCN', 'PCC_ID', 'Location_Name']]

,CCN,PCC_ID,Location_Name
3,455934.0,38.0,Northern Oaks Living and Rehabilitation Center
4,455934.0,169.0,Northern Oaks Living and Rehabilitation Center
6,675901.0,11.0,Cambridge Health and Rehabilitation Center
7,675901.0,178.0,Cambridge Health and Rehabilitation Center
9,675272.0,74.0,Willowbend Nursing and Rehabilitation Center
10,675272.0,171.0,Willowbend Nursing and Rehabilitation Center
13,675081.0,76.0,Golden Acres Living and Rehabilitation Center
14,675081.0,177.0,Golden Acres Living and Rehabilitation Center
19,675593.0,88.0,Wisteria Place
20,675593.0,170.0,Wisteria Place


In [80]:
map_df_clean = map_df.drop_duplicates('CCN', keep='first')
try:
    assert len(map_df) - 10 == len(map_df_clean)
except:
    print('check again')

In [81]:
map_df_clean['PCC_ID'] = map_df_clean['PCC_ID'].astype('int')

/var/folders/pg/m6t364yd5yl9c8mcnhmr67y40000gn/T/ipykernel_50049/3284487366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  map_df_clean['PCC_ID'] = map_df_clean['PCC_ID'].astype('int')


In [82]:
# rename columns
map_df_clean = map_df_clean.rename(columns={'PCC_ID': 'facilityid',
                            'CCN': 'provnum'})

In [83]:
map_for_merge = map_df_clean[['facilityid', 'provnum']]

In [84]:
map_for_merge

,facilityid,provnum
0,56,455754.0
1,72,455637.0
2,65,675689.0
3,38,455934.0
5,66,455745.0
...,...,...
287,268,465188.0
288,266,465003.0
290,267,135076.0
307,290,675611.0


In [85]:
ac_df[ac_df['facilityid']==56]['facility']

54    Northeast
Name: facility, dtype: object

In [86]:
ac_df.columns

Index(['facilityid', 'sourceid', 'facilityfk', 'facilitycode', 'consolidated',
       'consolidated_bpc_id', 'group', 'group_bpc_id', 'operations',
       'operations_bpc_id', 'region', 'region_bpc_id', 'company',
       'company_bpc_id', 'market', 'market_bpc_id', 'cluster',
       'cluster_bpc_id', 'facility', 'facility_bpc_id', 'facilitydetail',
       'facilitydetail_bpc_id', 'samestore', 'samestore_bpc_id', 'consops',
       'consops_bpc_id', 'bed size', 'availablebeds', 'lineofbusiness',
       'acquisitiondate', 'pcc_start', 'closedate', 'arstartdate',
       'geographyid', 'arresource', 'samestore1999', 'samestore2000',
       'samestore2001', 'samestore2002', 'samestore2003', 'samestore2004',
       'samestore2005', 'samestore2006', 'samestore2007', 'samestore2008',
       'samestore2009', 'samestore2010', 'samestore2011', 'samestore2012',
       'samestore2013', 'samestore2014', 'samestore2015', 'samestore2016',
       'samestore2017', 'samestore2018', 'samestore2019', 'sames

In [87]:
merge_df = pd.merge(
                    ac_df,
                    map_for_merge,
                    how = 'inner',
                    left_on = 'facilityid',
                    right_on = 'facilityid',
                    validate = 'one_to_one')

In [94]:
merge_df.dtypes
merge_df['acquisitiondate'] = merge_df['acquisitiondate'].astype(str)
merge_df

,provnum,facilityid,facility,acquisitiondate,region,market,cluster,aquisitiondate
0,56360.0,1,Arbor Glen,20010201,California,Touchstone 2.0,The Tough Ones,2001-02-01
1,465072.0,3,City Creek SNF UPL Total,20060901,Summit,Milestone North,Seal Team4,2006-09-01
2,55505.0,4,Arroyo Vista,20031001,California,SD United,AFC Cluster,2003-10-01
3,65321.0,5,Arvada,20090201,MidWest,Endura Market,Rocky Mountain A.I.R.,2009-02-01
4,55744.0,6,Atlantic,20020701,California,Momentum,GOAT,2002-07-01
...,...,...,...,...,...,...,...,...
226,675808.0,362,Windsor Rehabilitation and Healthcare,20210501,Texas,Keystone North Texas,TNT,2021-05-01
227,505315.0,363,Mira Vista Care Center,20210601,Summit,Pennant Washington,PNW,2021-06-01
228,505262.0,364,Shoreline Health and Rehabilitation,20210601,Summit,Pennant Washington,PNW,2021-06-01
229,505347.0,365,The Oaks at Lakewood,20210601,Summit,Pennant Washington,Sasquatch,2021-06-01


In [95]:
features = ['provnum', 'facilityid', 'facility', 'acquisitiondate', 'region', 'market', 'cluster']
merge_df = merge_df[features]
merge_df['provnum'] = merge_df['provnum'].astype(str)
merge_df['provnum'] = merge_df['provnum'].apply(lambda x: str(x).zfill(6))
merge_df['aquisitiondate'] = pd.to_datetime(merge_df['acquisitiondate'], format="%Y%m%d")
# merge_df['acquired_year'] = merge_df['acquisitiondate'].dt.year
# merge_df['acquired_month'] = merge_df['acquisitiondate'].dt.month
# merge_df['acquired_day'] = merge_df['acquisitiondate'].dt.day

/var/folders/pg/m6t364yd5yl9c8mcnhmr67y40000gn/T/ipykernel_50049/4056987791.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df['provnum'] = merge_df['provnum'].astype(str)
/var/folders/pg/m6t364yd5yl9c8mcnhmr67y40000gn/T/ipykernel_50049/4056987791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df['provnum'] = merge_df['provnum'].apply(lambda x: str(x).zfill(6))


In [72]:
merge_df

,provnum,facilityid,facility,acquisitiondate,region,market,cluster,aquisitiondate
0,56360.0,1,Arbor Glen,20010201,California,Touchstone 2.0,The Tough Ones,2001-02-01
1,465072.0,3,City Creek SNF UPL Total,20060901,Summit,Milestone North,Seal Team4,2006-09-01
2,55505.0,4,Arroyo Vista,20031001,California,SD United,AFC Cluster,2003-10-01
3,65321.0,5,Arvada,20090201,MidWest,Endura Market,Rocky Mountain A.I.R.,2009-02-01
4,55744.0,6,Atlantic,20020701,California,Momentum,GOAT,2002-07-01
...,...,...,...,...,...,...,...,...
226,675808.0,362,Windsor Rehabilitation and Healthcare,20210501,Texas,Keystone North Texas,TNT,2021-05-01
227,505315.0,363,Mira Vista Care Center,20210601,Summit,Pennant Washington,PNW,2021-06-01
228,505262.0,364,Shoreline Health and Rehabilitation,20210601,Summit,Pennant Washington,PNW,2021-06-01
229,505347.0,365,The Oaks at Lakewood,20210601,Summit,Pennant Washington,Sasquatch,2021-06-01


In [ ]:
merge_df.to_csv('all/acquisition_dates.csv', index=False)

In [57]:
merge_df.dtypes

facilityid        int64
sourceid          int64
facilityfk        int64
facilitycode     object
consolidated     object
                 ...   
isbpci             bool
iscjr              bool
ishcr              bool
entitytype       object
provnum         float64
Length: 63, dtype: object